# Импорты

In [61]:
import numpy as np
import pandas as pd
import html
import string

import lightgbm

# Очистка трейна

In [ ]:
# data_train = pd.read_json('ranking_train.jsonl', lines=True)
# new_rows = []
# for index, row in data_train.iterrows():
#     new_row = {'text': row['text']}
#     for i in range(len(row['comments'])):
#         new_row['comments_text_' + str(i)] = row['comments'][i]['text']
#     new_rows.append(new_row)

# pd.DataFrame(new_rows).to_csv('ranking_train.tsv', sep="\t", index=False)

In [3]:
# train_data_tsv = pd.read_csv('ranking_train.tsv', sep="\t")
# train_data_tsv[:10]

,text,comments_text_0,comments_text_1,comments_text_2,comments_text_3,comments_text_4
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,There will invariably be those who don't see t...,For me school is a way to be connected to what...,I guess it really depends on how hungry you ar...,I know pollground decided to go back to school...
1,CBS acquires last.fm for $280m,It will be curious to see where this heads in ...,Does this mean that there's now a big-name com...,Also on BBC News: <url> .Nice to see a Lond...,I don't understand what they do that is worth ...,sold out too cheaply. given their leadership p...
2,How Costco Became the Anti-Wal-Mart,I really hate it when people falsely claim tha...,"I love Costco. For me, the best quote of the ...","""But Mr. Sinegal warned that if Costco increas...",While Costco does treat its employees a little...,"I'd like to know more about their ""don't adver..."
3,"Fortune Favors Big Turds | Screw The Money, Th...","His real point is that something can be ""simpl...","So I'm thinking, ""Well, I've never heard of th...",I've never seen a single blog post wander in s...,It was a RANT on a personal blog. It's there t...,"In the eternal words of the Geico Caveman..""Wh..."
4,StartupWeekend: 70 Founders Create One Company...,Looks like someone hasn't read The Mythical Ma...,Any chances for recreating this in a telecommu...,"You know, if they started with 70 and eliminat...",And what did you do this weekend?I'd love to h...,Very interesting. I would join an event like t...
5,Younger job seekers hold the advantage (becaus...,I smell bullshit. Note from the article:> Comp...,"Not a bad read, although I wonder what reality...","More than anything, I think this is about all ...",Some even have their parents in the room for a...,One thing that bugs me about SlapVid is that i...
6,Four Pownce Invites,Ok these are out the door. I have a few more ...,sgoraya at gmail.com\nthx!,<email> .,<email> Thanks.,kyle.bolton at gmail
7,LFMs and LFSPs (Michael Vanier via PG),I'd forgotten how well written that was. It's...,"Perfection is achieved, not when there is noth...","To get wider use of LFSPs, it would be importa...",I totally understand people's complaints about...,for (i = 0; i < N; i++)Odd example to use. It'...
8,The Virtues of a Second Screen,I guess I'm one of the few people who has trie...,"I didn't look at the date, but when I saw the ...",I am a big believer in 2 screens as well. I h...,"Bah. A 30"" is vastly superior to two screens. ...",Why is it posted here?
9,"Startup Weekend failed, but lessons learned",70 days and 2 people would work much better th...,"They learned a lesson indeed: ""Development is ...",Is anyone else confused by the premise of star...,> The Java platform was selected/Nuff\n,This is the first I'd heard of startupweekend....


In [7]:
data = pd.read_json('ranking_train.jsonl', lines=True)
new_rows = []
for index, row in data.iterrows():
    for comment in row['comments']:
        new_row = {'text': row['text'], 'comments_text': comment['text'], 'comments_score': comment['score']}
        new_rows.append(new_row)

In [51]:
data = pd.DataFrame(new_rows)
data = data.reset_index(drop=True)
data

,text,comments_text,comments_score
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0
1,How many summer Y Combinator fundees decided n...,There will invariably be those who don't see t...,1
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,2
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,3
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,4
...,...,...,...
440530,Pay your rent with a Credit or Debit card. No ...,Most major banks offer a service called &#x27;...,0
440531,Pay your rent with a Credit or Debit card. No ...,"It costs 3.25%, or $74.25 for the example of $...",1
440532,Pay your rent with a Credit or Debit card. No ...,As many other comments have pointed out almost...,2
440533,Pay your rent with a Credit or Debit card. No ...,My apartment building uses Yapstone&#x27;s Ren...,3


In [32]:
def preprocessing(data):

    for column in data.columns:
        #заменяем коды на символы
        data[column] = data[column].apply(html.unescape)

        #заменяем url'ы на токен 1url1
        data[column] = data[column].str.replace(
        r"(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])",
        '1url1',
        regex=True)

        #заменяем email'ы на токен 1email1
        data[column] = data[column].str.replace(
            r"(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*)@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])",
            '1email1',
            regex=True)

        #убираем ссылки
        data[column] = data[column].str.replace(r'\[\d\]', '', regex=True)

        # #понижаем регистр
        data[column] = data[column].apply(lambda x: x.lower())

        #избавляемся от лишних знаков пунктуации
        data[column] = data[column].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation + "\n")))

        #делаем обратную замену на <token>
        data[column] = data[column].str.replace('1url1', ' <url> ')
        data[column] = data[column].str.replace('1email1', ' <email> ')

    return data

In [53]:
data.iloc[:, :2] = preprocessing(data.iloc[:, :2])
data

,text,comments_text,comments_score
0,how many summer y combinator fundees decided n...,going back to school is not identical with giv...,0
1,how many summer y combinator fundees decided n...,there will invariably be those who dont see th...,1
2,how many summer y combinator fundees decided n...,for me school is a way to be connected to what...,2
3,how many summer y combinator fundees decided n...,i guess it really depends on how hungry you ar...,3
4,how many summer y combinator fundees decided n...,i know pollground decided to go back to school...,4
...,...,...,...
440530,pay your rent with a credit or debit card no l...,most major banks offer a service called bill p...,0
440531,pay your rent with a credit or debit card no l...,it costs 325 or 7425 for the example of 2300 s...,1
440532,pay your rent with a credit or debit card no l...,as many other comments have pointed out almost...,2
440533,pay your rent with a credit or debit card no l...,my apartment building uses yapstones rentpayme...,3


# Train-Val Split

In [54]:
from sklearn.model_selection import GroupShuffleSplit

splitter = GroupShuffleSplit(test_size=0.2, n_splits=2, random_state=42)
split = splitter.split(data, groups=data['text'])
train_inds, val_inds = next(split)

train = data.iloc[train_inds]
val = data.iloc[val_inds]

In [83]:
X_train_raw = train['text'] + train['comments_text']
y_train = np.array(train['comments_score'])

X_val_raw = val['text'] + val['comments_text']
y_val = np.array(val['comments_score'])

# Векторизация

In [48]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

import gensim.downloader
from tqdm import tqdm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
import vector_space_models_utils
from vector_space_models_utils import rank_comments
from vector_space_models_utils import Embeddings_text_to_vector
from vector_space_models_utils import get_predicted_order
from vector_space_models_utils import hits_score
from vector_space_models_utils import dcg_score

In [46]:
stopWords = set(stopwords.words('english'))

In [47]:
class NLTK_Tokenizer:
    def __init__(self, delete_punctuation=False, delete_stop_words=False):
        self.delete_punctuation = delete_punctuation
        self.delete_stop_words = delete_stop_words
    def tokenize(self, text, ):
        if self.delete_punctuation:
            text = text.translate(str.maketrans('', '', string.punctuation))
        tokens = word_tokenize(text.lower())
        if self.delete_stop_words:
            return [token for token in tokens if token not in stopWords]
        return tokens

nltk_tokenizer = NLTK_Tokenizer()

In [49]:
embeddings_list = [('glove-wiki-gigaword-50', 50),
                   ('glove-wiki-gigaword-100', 100),
                   ('glove-wiki-gigaword-200', 200),
                   ('glove-twitter-25', 25),
                   ('glove-twitter-50', 50),
                   ('glove-twitter-100', 100)]

In [79]:
def text_to_vec(text, embeddings, tokenizer, dim=200, aggregation='mean', verbose=False):
    """
        text: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    tokens = tokenizer.tokenize(text)

    result = np.zeros(dim)

    if verbose:
        print('Initial text:', text)
        print('Tokens:', tokens)

    if aggregation == 'mean':
        iterator = 0

        for word in tokens:
            if word in embeddings:
                result += embeddings[word]
                iterator += 1
                if verbose and dim > 5:
                    print('Word and its embedding:', word, embeddings[word][:5])

        if iterator != 0:
            result = result / iterator
    else:
        raise AttributeError(f'No aggregation type {aggregation}, please, use other')


    return result

In [80]:
def transform_data(embeddings, y, vector_dim):
    idx = []
    for i in range(int(len(embeddings) / 5)):
        idx.extend([i] * 5)

    embeddings_df = pd.DataFrame(embeddings, columns=[i for i in range(1, vector_dim+1)])
    embeddings_df['score'] = y
    embeddings_df['qid'] = idx

    qids = embeddings_df.groupby('qid')['qid'].count().to_numpy()
    X = embeddings_df.drop(['score', 'qid'], axis = 1)
    #y = embeddings_df['score'].astype(int)

    return X, qids

# Сравнение различных эмбеддингов на LambdaMART модели

In [91]:
for embedding_name, vector_dim in tqdm(embeddings_list):

    print()

    current_embeddings = gensim.downloader.load(embedding_name)
    current_tokenizer = NLTK_Tokenizer(delete_punctuation=True, delete_stop_words=True)

    print()
    print(embedding_name)
    print()

    # Создаем эмбеддинги для датасета
    train_emb = [text_to_vec(row, current_embeddings, current_tokenizer, vector_dim) for row in X_train_raw]
    val_emb = [text_to_vec(row, current_embeddings, current_tokenizer, vector_dim) for row in X_val_raw]

    # Подготавливаем данные для подачи в модель
    X_train, qids_train = transform_data(train_emb, y_train, vector_dim)
    X_val, qids_val = transform_data(val_emb, y_val, vector_dim)

    # Создаем модель
    ranker = lightgbm.LGBMRanker(
        objective='lambdarank',
        boosting_type='gbdt',
        n_estimators=5,
        importance_type='gain',
        metric='ndcg',
        num_leaves=10,
        learning_rate=0.05,
        max_depth=-1,
        label_gain=[i for i in range(5)]
    )

    # Обучаем модель
    ranker.fit(
        X=X_train,
        y=y_train,
        group=qids_train,
        eval_set=[(X_val, y_val)],
        eval_group=[qids_val],
        eval_at=[1, 3, 5]
    )

  0%|          | 0/6 [00:00<?, ?it/s]



glove-wiki-gigaword-50

[1]	valid_0's ndcg@1: 0.453291	valid_0's ndcg@3: 0.576642	valid_0's ndcg@5: 0.784891
[2]	valid_0's ndcg@1: 0.556738	valid_0's ndcg@3: 0.656875	valid_0's ndcg@5: 0.825513
[3]	valid_0's ndcg@1: 0.58817	valid_0's ndcg@3: 0.680489	valid_0's ndcg@5: 0.83778
[4]	valid_0's ndcg@1: 0.601589	valid_0's ndcg@3: 0.694311	valid_0's ndcg@5: 0.843943
[5]	valid_0's ndcg@1: 0.609617	valid_0's ndcg@3: 0.700847	valid_0's ndcg@5: 0.847255


 17%|█▋        | 1/6 [05:51<29:18, 351.61s/it]



glove-wiki-gigaword-100

[1]	valid_0's ndcg@1: 0.451844	valid_0's ndcg@3: 0.578905	valid_0's ndcg@5: 0.785632
[2]	valid_0's ndcg@1: 0.564936	valid_0's ndcg@3: 0.665218	valid_0's ndcg@5: 0.82949
[3]	valid_0's ndcg@1: 0.597943	valid_0's ndcg@3: 0.690997	valid_0's ndcg@5: 0.842469
[4]	valid_0's ndcg@1: 0.608823	valid_0's ndcg@3: 0.700837	valid_0's ndcg@5: 0.847122


 33%|███▎      | 2/6 [12:34<25:27, 381.88s/it]

...

[4]	valid_0's ndcg@1: 0.585589	valid_0's ndcg@3: 0.677775	valid_0's ndcg@5: 0.83635
[5]	valid_0's ndcg@1: 0.59322	valid_0's ndcg@3: 0.685552	valid_0's ndcg@5: 0.839962


glove-twitter-50

[1]	valid_0's ndcg@1: 0.460823	valid_0's ndcg@3: 0.571338	valid_0's ndcg@5: 0.784834
[2]	valid_0's ndcg@1: 0.564071	valid_0's ndcg@3: 0.656135	valid_0's ndcg@5: 0.826406
[3]	valid_0's ndcg@1: 0.588894	valid_0's ndcg@3: 0.682349	valid_0's ndcg@5: 0.838365
[4]	valid_0's ndcg@1: 0.600624	valid_0's ndcg@3: 0.691592	valid_0's ndcg@5: 0.842959


 83%|████████▎ | 5/6 [33:26<06:39, 399.28s/it]

[5]	valid_0's ndcg@1: 0.611376	valid_0's ndcg@3: 0.702598	valid_0's ndcg@5: 0.848127


glove-twitter-100

[1]	valid_0's ndcg@1: 0.473816	valid_0's ndcg@3: 0.585281	valid_0's ndcg@5: 0.79101
[2]	valid_0's ndcg@1: 0.575574	valid_0's ndcg@3: 0.664197	valid_0's ndcg@5: 0.830895
[3]	valid_0's ndcg@1: 0.596071	valid_0's ndcg@3: 0.687483	valid_0's ndcg@5: 0.841112
[4]	valid_0's ndcg@1: 0.608723	valid_0's ndcg@3: 0.699835	valid_0's ndcg@5: 0.846792


100%|██████████| 6/6 [41:23<00:00, 413.88s/it]

[5]	valid_0's ndcg@1: 0.614709	valid_0's ndcg@3: 0.707019	valid_0's ndcg@5: 0.849991


## ('glove-wiki-gigaword-50', 50)

In [81]:
embedding_name = embeddings_list[0][0]
vector_dim = embeddings_list[0][1]

current_embeddings = gensim.downloader.load(embedding_name)
current_tokenizer = NLTK_Tokenizer(delete_punctuation=True, delete_stop_words=True)

# Создаем эмбеддинги для датасета
train_emb = [text_to_vec(row, current_embeddings, current_tokenizer, vector_dim) for row in X_train]
val_emb = [text_to_vec(row, current_embeddings, current_tokenizer, vector_dim) for row in X_val]

# Подготавливаем данные для подачи в модель
X_train, qids_train = transform_data(train_emb, y_train, vector_dim)
X_val, qids_val = transform_data(val_emb, y_val, vector_dim)

# Создаем модель
ranker = lightgbm.LGBMRanker(
    objective='lambdarank',
    boosting_type='gbdt',
    n_estimators=5,
    importance_type='gain',
    metric='ndcg',
    num_leaves=10,
    learning_rate=0.05,
    max_depth=-1,
    label_gain=[i for i in range(5)]
)

# Обучаем модель
ranker.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_val, y_val)],
    eval_group=[qids_val],
    eval_at=[1, 3, 5]
)

[1]	valid_0's ndcg@1: 0.453291	valid_0's ndcg@3: 0.576642	valid_0's ndcg@5: 0.784891
[2]	valid_0's ndcg@1: 0.556738	valid_0's ndcg@3: 0.656875	valid_0's ndcg@5: 0.825513
[3]	valid_0's ndcg@1: 0.58817	valid_0's ndcg@3: 0.680489	valid_0's ndcg@5: 0.83778
[4]	valid_0's ndcg@1: 0.601589	valid_0's ndcg@3: 0.694311	valid_0's ndcg@5: 0.843943
[5]	valid_0's ndcg@1: 0.609617	valid_0's ndcg@3: 0.700847	valid_0's ndcg@5: 0.847255


LGBMRanker(importance_type='gain', label_gain=[0, 1, 2, 3, 4],
           learning_rate=0.05, metric='ndcg', n_estimators=5, num_leaves=10,
           objective='lambdarank')

## ('glove-wiki-gigaword-100', 100)

In [84]:
embedding_name = embeddings_list[1][0]
vector_dim = embeddings_list[1][1]

current_embeddings = gensim.downloader.load(embedding_name)
current_tokenizer = NLTK_Tokenizer(delete_punctuation=True, delete_stop_words=True)

# Создаем эмбеддинги для датасета
train_emb = [text_to_vec(row, current_embeddings, current_tokenizer, vector_dim) for row in X_train_raw]
val_emb = [text_to_vec(row, current_embeddings, current_tokenizer, vector_dim) for row in X_val_raw]

# Подготавливаем данные для подачи в модель
X_train, qids_train = transform_data(train_emb, y_train, vector_dim)
X_val, qids_val = transform_data(val_emb, y_val, vector_dim)

# Создаем модель
ranker = lightgbm.LGBMRanker(
    objective='lambdarank',
    boosting_type='gbdt',
    n_estimators=5,
    importance_type='gain',
    metric='ndcg',
    num_leaves=10,
    learning_rate=0.05,
    max_depth=-1,
    label_gain=[i for i in range(5)]
)

# Обучаем модель
ranker.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_val, y_val)],
    eval_group=[qids_val],
    eval_at=[1, 3, 5]
)

[1]	valid_0's ndcg@1: 0.451844	valid_0's ndcg@3: 0.578905	valid_0's ndcg@5: 0.785632
[2]	valid_0's ndcg@1: 0.564936	valid_0's ndcg@3: 0.665218	valid_0's ndcg@5: 0.82949
[3]	valid_0's ndcg@1: 0.597943	valid_0's ndcg@3: 0.690997	valid_0's ndcg@5: 0.842469
[4]	valid_0's ndcg@1: 0.608823	valid_0's ndcg@3: 0.700837	valid_0's ndcg@5: 0.847122
[5]	valid_0's ndcg@1: 0.617149	valid_0's ndcg@3: 0.707373	valid_0's ndcg@5: 0.850276


LGBMRanker(importance_type='gain', label_gain=[0, 1, 2, 3, 4],
           learning_rate=0.05, metric='ndcg', n_estimators=5, num_leaves=10,
           objective='lambdarank')

## ('glove-wiki-gigaword-200', 200)

In [85]:
embedding_name = embeddings_list[2][0]
vector_dim = embeddings_list[2][1]

current_embeddings = gensim.downloader.load(embedding_name)
current_tokenizer = NLTK_Tokenizer(delete_punctuation=True, delete_stop_words=True)

# Создаем эмбеддинги для датасета
train_emb = [text_to_vec(row, current_embeddings, current_tokenizer, vector_dim) for row in X_train_raw]
val_emb = [text_to_vec(row, current_embeddings, current_tokenizer, vector_dim) for row in X_val_raw]

# Подготавливаем данные для подачи в модель
X_train, qids_train = transform_data(train_emb, y_train, vector_dim)
X_val, qids_val = transform_data(val_emb, y_val, vector_dim)

# Создаем модель
ranker = lightgbm.LGBMRanker(
    objective='lambdarank',
    boosting_type='gbdt',
    n_estimators=5,
    importance_type='gain',
    metric='ndcg',
    num_leaves=10,
    learning_rate=0.05,
    max_depth=-1,
    label_gain=[i for i in range(5)]
)

# Обучаем модель
ranker.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_val, y_val)],
    eval_group=[qids_val],
    eval_at=[1, 3, 5]
)

[=================================================-] 99.0% 249.5/252.1MB downloaded
[1]	valid_0's ndcg@1: 0.450397	valid_0's ndcg@3: 0.583718	valid_0's ndcg@5: 0.786575
[2]	valid_0's ndcg@1: 0.569362	valid_0's ndcg@3: 0.666479	valid_0's ndcg@5: 0.830534
[3]	valid_0's ndcg@1: 0.60773	valid_0's ndcg@3: 0.694624	valid_0's ndcg@5: 0.844981
[4]	valid_0's ndcg@1: 0.624128	valid_0's ndcg@3: 0.708904	valid_0's ndcg@5: 0.851969
[5]	valid_0's ndcg@1: 0.627759	valid_0's ndcg@3: 0.714537	valid_0's ndcg@5: 0.854429


LGBMRanker(importance_type='gain', label_gain=[0, 1, 2, 3, 4],
           learning_rate=0.05, metric='ndcg', n_estimators=5, num_leaves=10,
           objective='lambdarank')

## ('glove-twitter-25', 25)

In [86]:
embedding_name = embeddings_list[3][0]
vector_dim = embeddings_list[3][1]

current_embeddings = gensim.downloader.load(embedding_name)
current_tokenizer = NLTK_Tokenizer(delete_punctuation=True, delete_stop_words=True)

# Создаем эмбеддинги для датасета
train_emb = [text_to_vec(row, current_embeddings, current_tokenizer, vector_dim) for row in X_train_raw]
val_emb = [text_to_vec(row, current_embeddings, current_tokenizer, vector_dim) for row in X_val_raw]

# Подготавливаем данные для подачи в модель
X_train, qids_train = transform_data(train_emb, y_train, vector_dim)
X_val, qids_val = transform_data(val_emb, y_val, vector_dim)

# Создаем модель
ranker = lightgbm.LGBMRanker(
    objective='lambdarank',
    boosting_type='gbdt',
    n_estimators=5,
    importance_type='gain',
    metric='ndcg',
    num_leaves=10,
    learning_rate=0.05,
    max_depth=-1,
    label_gain=[i for i in range(5)]
)

# Обучаем модель
ranker.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_val, y_val)],
    eval_group=[qids_val],
    eval_at=[1, 3, 5]
)

[==================================================] 100.0% 104.8/104.8MB downloaded
[1]	valid_0's ndcg@1: 0.459844	valid_0's ndcg@3: 0.568761	valid_0's ndcg@5: 0.78393
[2]	valid_0's ndcg@1: 0.544638	valid_0's ndcg@3: 0.635997	valid_0's ndcg@5: 0.817476
[3]	valid_0's ndcg@1: 0.566638	valid_0's ndcg@3: 0.666379	valid_0's ndcg@5: 0.830066
[4]	valid_0's ndcg@1: 0.585589	valid_0's ndcg@3: 0.677775	valid_0's ndcg@5: 0.83635
[5]	valid_0's ndcg@1: 0.59322	valid_0's ndcg@3: 0.685552	valid_0's ndcg@5: 0.839962


LGBMRanker(importance_type='gain', label_gain=[0, 1, 2, 3, 4],
           learning_rate=0.05, metric='ndcg', n_estimators=5, num_leaves=10,
           objective='lambdarank')

## ('glove-twitter-50', 50)

In [87]:
embedding_name = embeddings_list[4][0]
vector_dim = embeddings_list[4][1]

current_embeddings = gensim.downloader.load(embedding_name)
current_tokenizer = NLTK_Tokenizer(delete_punctuation=True, delete_stop_words=True)

# Создаем эмбеддинги для датасета
train_emb = [text_to_vec(row, current_embeddings, current_tokenizer, vector_dim) for row in X_train_raw]
val_emb = [text_to_vec(row, current_embeddings, current_tokenizer, vector_dim) for row in X_val_raw]

# Подготавливаем данные для подачи в модель
X_train, qids_train = transform_data(train_emb, y_train, vector_dim)
X_val, qids_val = transform_data(val_emb, y_val, vector_dim)

# Создаем модель
ranker = lightgbm.LGBMRanker(
    objective='lambdarank',
    boosting_type='gbdt',
    n_estimators=5,
    importance_type='gain',
    metric='ndcg',
    num_leaves=10,
    learning_rate=0.05,
    max_depth=-1,
    label_gain=[i for i in range(5)]
)

# Обучаем модель
ranker.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_val, y_val)],
    eval_group=[qids_val],
    eval_at=[1, 3, 5]
)

[================================================--] 97.1% 193.8/199.5MB downloaded[1]	valid_0's ndcg@1: 0.460823	valid_0's ndcg@3: 0.571338	valid_0's ndcg@5: 0.784834
[2]	valid_0's ndcg@1: 0.564071	valid_0's ndcg@3: 0.656135	valid_0's ndcg@5: 0.826406
[3]	valid_0's ndcg@1: 0.588894	valid_0's ndcg@3: 0.682349	valid_0's ndcg@5: 0.838365
[4]	valid_0's ndcg@1: 0.600624	valid_0's ndcg@3: 0.691592	valid_0's ndcg@5: 0.842959
[5]	valid_0's ndcg@1: 0.611376	valid_0's ndcg@3: 0.702598	valid_0's ndcg@5: 0.848127


LGBMRanker(importance_type='gain', label_gain=[0, 1, 2, 3, 4],
           learning_rate=0.05, metric='ndcg', n_estimators=5, num_leaves=10,
           objective='lambdarank')

## ('glove-twitter-100', 100)

In [88]:
embedding_name = embeddings_list[5][0]
vector_dim = embeddings_list[5][1]

current_embeddings = gensim.downloader.load(embedding_name)
current_tokenizer = NLTK_Tokenizer(delete_punctuation=True, delete_stop_words=True)

# Создаем эмбеддинги для датасета
train_emb = [text_to_vec(row, current_embeddings, current_tokenizer, vector_dim) for row in X_train_raw]
val_emb = [text_to_vec(row, current_embeddings, current_tokenizer, vector_dim) for row in X_val_raw]

# Подготавливаем данные для подачи в модель
X_train, qids_train = transform_data(train_emb, y_train, vector_dim)
X_val, qids_val = transform_data(val_emb, y_val, vector_dim)

# Создаем модель
ranker = lightgbm.LGBMRanker(
    objective='lambdarank',
    boosting_type='gbdt',
    n_estimators=5,
    importance_type='gain',
    metric='ndcg',
    num_leaves=10,
    learning_rate=0.05,
    max_depth=-1,
    label_gain=[i for i in range(5)]
)

# Обучаем модель
ranker.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_val, y_val)],
    eval_group=[qids_val],
    eval_at=[1, 3, 5]
)

[=================================================-] 99.9% 386.7/387.1MB downloaded
[1]	valid_0's ndcg@1: 0.473816	valid_0's ndcg@3: 0.585281	valid_0's ndcg@5: 0.79101
[2]	valid_0's ndcg@1: 0.575574	valid_0's ndcg@3: 0.664197	valid_0's ndcg@5: 0.830895
[3]	valid_0's ndcg@1: 0.596071	valid_0's ndcg@3: 0.687483	valid_0's ndcg@5: 0.841112
[4]	valid_0's ndcg@1: 0.608723	valid_0's ndcg@3: 0.699835	valid_0's ndcg@5: 0.846792
[5]	valid_0's ndcg@1: 0.614709	valid_0's ndcg@3: 0.707019	valid_0's ndcg@5: 0.849991


LGBMRanker(importance_type='gain', label_gain=[0, 1, 2, 3, 4],
           learning_rate=0.05, metric='ndcg', n_estimators=5, num_leaves=10,
           objective='lambdarank')

# Оптимизация гиперпараметров

In [92]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.1 MB/s eta 0:00:00


In [93]:
embedding_name = embeddings_list[2][0]
vector_dim = embeddings_list[2][1]

current_embeddings = gensim.downloader.load(embedding_name)
current_tokenizer = NLTK_Tokenizer(delete_punctuation=True, delete_stop_words=True)

train_emb = [text_to_vec(row, current_embeddings, current_tokenizer, vector_dim) for row in X_train_raw]
y = y_train

idx = []
for i in range(int(len(train_emb) / 5)):
    idx.extend([i] * 5)

embeddings_df = pd.DataFrame(train_emb, columns=[i for i in range(1, vector_dim+1)])
embeddings_df['score'] = y
embeddings_df['qid'] = idx

qids = embeddings_df.groupby('qid')['qid'].count().to_numpy()
X = embeddings_df.drop(['score'], axis = 1)

In [94]:
import lightgbm as lgb
import optuna

def objective(trial, X, y):
    params = {
        'objective': 'lambdarank',
        'boosting_type': 'gbdt',
        'n_estimators': 5,
        'importance_type': 'gain',
        'metric': 'ndcg',
        'num_leaves': trial.suggest_int('num_leaves', 2, 50),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', -1, 10),
        'label_gain': [i for i in range(5)]
    }

    cv = GroupShuffleSplit(test_size=0.2, n_splits=2, random_state=42)

    scores = []

    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y, groups=X['qid'])):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        ranker = lgb.LGBMRanker(**params)

        qids_train = X_train.groupby('qid')['qid'].count().to_numpy()
        qids_test = X_test.groupby('qid')['qid'].count().to_numpy()

        X_train = X_train.drop(['qid'], axis = 1)
        X_test = X_test.drop(['qid'], axis = 1)

        ranker.fit(
            X=X_train,
            y=y_train,
            group=qids_train,
            eval_set=[(X_test, y_test)],
            eval_group=[qids_test],
            eval_at=[1, 3, 5]
        )

        # Calculate NDCG score on the validation set
        ndcg = ranker.best_score_['valid_0']['ndcg@5']
        scores.append(ndcg)

    return np.mean(scores)

study = optuna.create_study(direction='maximize', study_name='LGBMRanker')
func = lambda trial: objective(trial, X=X, y=y)
study.optimize(func, n_trials=50)

print('Best trial:')
best_trial = study.best_trial
print('NDCG: {}'.format(best_trial.value))
print('Hyperparameters: {}'.format(best_trial.params))

[I 2023-06-24 02:17:30,431] A new study created in memory with name: LGBMRanker
<ipython-input-94-df06586a13cd>:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),


[1]	valid_0's ndcg@1: 0.520093	valid_0's ndcg@3: 0.616759	valid_0's ndcg@5: 0.807933
[2]	valid_0's ndcg@1: 0.598549	valid_0's ndcg@3: 0.6816	valid_0's ndcg@5: 0.83981
[3]	valid_0's ndcg@1: 0.617472	valid_0's ndcg@3: 0.703136	valid_0's ndcg@5: 0.849126
[4]	valid_0's ndcg@1: 0.624672	valid_0's ndcg@3: 0.710577	valid_0's ndcg@5: 0.852701
[5]	valid_0's ndcg@1: 0.62719	valid_0's ndcg@3: 0.714716	valid_0's ndcg@5: 0.854343
[1]	valid_0's ndcg@1: 0.518869	valid_0's ndcg@3: 0.618025	valid_0's ndcg@5: 0.807903
[2]	valid_0's ndcg@1: 0.596811	valid_0's ndcg@3: 0.684152	valid_0's ndcg@5: 0.840164
[3]	valid_0's ndcg@1: 0.613712	valid_0's ndcg@3: 0.701198	valid_0's ndcg@5: 0.848045
[4]	valid_0's ndcg@1: 0.620061	valid_0's ndcg@3: 0.707468	valid_0's ndcg@5: 0.85086


[I 2023-06-24 02:18:11,789] Trial 0 finished with value: 0.8542439150390133 and parameters: {'num_leaves': 19, 'learning_rate': 0.03034246577425776, 'max_depth': 10}. Best is trial 0 with value: 0.8542439150390133.


[5]	valid_0's ndcg@1: 0.627421	valid_0's ndcg@3: 0.714053	valid_0's ndcg@5: 0.854145


<ipython-input-94-df06586a13cd>:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),


...

[5]	valid_0's ndcg@1: 0.573615	valid_0's ndcg@3: 0.62433	valid_0's ndcg@5: 0.818929


<ipython-input-94-df06586a13cd>:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),


[1]	valid_0's ndcg@1: 0.183355	valid_0's ndcg@3: 0.332443	valid_0's ndcg@5: 0.668818
[2]	valid_0's ndcg@1: 0.298858	valid_0's ndcg@3: 0.402918	valid_0's ndcg@5: 0.708597
[3]	valid_0's ndcg@1: 0.371036	valid_0's ndcg@3: 0.455565	valid_0's ndcg@5: 0.736114
[4]	valid_0's ndcg@1: 0.432078	valid_0's ndcg@3: 0.498921	valid_0's ndcg@5: 0.758898
[5]	valid_0's ndcg@1: 0.472405	valid_0's ndcg@3: 0.530285	valid_0's ndcg@5: 0.774722
[1]	valid_0's ndcg@1: 0.168529	valid_0's ndcg@3: 0.323784	valid_0's ndcg@5: 0.663856
[2]	valid_0's ndcg@1: 0.288146	valid_0's ndcg@3: 0.395031	valid_0's ndcg@5: 0.704486
[3]	valid_0's ndcg@1: 0.374104	valid_0's ndcg@3: 0.452523	valid_0's ndcg@5: 0.735611
[4]	valid_0's ndcg@1: 0.425374	valid_0's ndcg@3: 0.492152	valid_0's ndcg@5: 0.755804


[I 2023-06-24 02:51:48,533] Trial 49 finished with value: 0.7734672778693916 and parameters: {'num_leaves': 47, 'learning_rate': 0.05779104016866895, 'max_depth': 1}. Best is trial 33 with value: 0.8594888026805935.


[5]	valid_0's ndcg@1: 0.467759	valid_0's ndcg@3: 0.524164	valid_0's ndcg@5: 0.772213
Best trial:
NDCG: 0.8594888026805935
Hyperparameters: {'num_leaves': 44, 'learning_rate': 0.06518640195332566, 'max_depth': -1}


# Валидация

In [109]:
splitter = GroupShuffleSplit(test_size=0.2, n_splits=2, random_state=42)
split = splitter.split(data, groups=data['text'])
train_inds, val_inds = next(split)

train = data.iloc[train_inds]
val = data.iloc[val_inds]

In [110]:
X_train_raw = train['text'] + train['comments_text']
y_train = np.array(train['comments_score'])

X_val_raw = val['text'] + val['comments_text']
y_val = np.array(val['comments_score'])

In [111]:
embedding_name = embeddings_list[2][0]
vector_dim = embeddings_list[2][1]

best_embedding = gensim.downloader.load(embedding_name)
best_tokenizer = NLTK_Tokenizer(delete_punctuation=True, delete_stop_words=True)

train_emb = [text_to_vec(row, best_embedding, best_tokenizer, vector_dim) for row in X_train_raw]
val_emb = [text_to_vec(row, best_embedding, best_tokenizer, vector_dim) for row in X_val_raw]

In [112]:
idx = []
for i in range(int(len(train_emb) / 5)):
    idx.extend([i] * 5)

embeddings_df = pd.DataFrame(train_emb, columns=[i for i in range(1, vector_dim+1)])
embeddings_df['score'] = y_train
embeddings_df['qid'] = idx

qids_train = embeddings_df.groupby('qid')['qid'].count().to_numpy()
X = embeddings_df.drop(['score'], axis = 1)
y = y_train

splitter = GroupShuffleSplit(test_size=0.2, n_splits=2, random_state=42)
split = splitter.split(X, y, groups=X['qid'])
train_idx, test_idx = next(split)

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

qids_train = X_train.groupby('qid')['qid'].count().to_numpy()
qids_test = X_test.groupby('qid')['qid'].count().to_numpy()

In [113]:
best_hyper = best_trial.params

ranker = lightgbm.LGBMRanker(
                    objective='lambdarank',
                    boosting_type='gbdt',
                    n_estimators=5,
                    importance_type='gain',
                    metric='ndcg',
                    num_leaves=best_hyper['num_leaves'],
                    learning_rate=best_hyper['learning_rate'],
                    max_depth=best_hyper['max_depth'],
                    label_gain=[i for i in range(max(y_train.max(), y_test.max()) + 1)])

ranker.fit(
      X=X_train,
      y=y_train,
      group=qids_train,
      eval_set=[(X_test, y_test)],
      eval_group=[qids_test],
      eval_at=[1, 3, 5])

[1]	valid_0's ndcg@1: 0.541906	valid_0's ndcg@3: 0.656678	valid_0's ndcg@5: 0.823057
[2]	valid_0's ndcg@1: 0.606689	valid_0's ndcg@3: 0.703323	valid_0's ndcg@5: 0.847909
[3]	valid_0's ndcg@1: 0.625931	valid_0's ndcg@3: 0.715774	valid_0's ndcg@5: 0.855023
[4]	valid_0's ndcg@1: 0.635082	valid_0's ndcg@3: 0.72265	valid_0's ndcg@5: 0.858352
[5]	valid_0's ndcg@1: 0.635135	valid_0's ndcg@3: 0.725211	valid_0's ndcg@5: 0.858996


LGBMRanker(importance_type='gain', label_gain=[0, 1, 2, 3, 4],
           learning_rate=0.06518640195332566, metric='ndcg', n_estimators=5,
           num_leaves=44, objective='lambdarank')

In [114]:
idx = []
for i in range(int(len(val_emb) / 5)):
    idx.extend([i] * 5)

val_embeddings_df = pd.DataFrame(val_emb, columns=[i for i in range(1, vector_dim+1)])
val_embeddings_df['score'] = y_val
val_embeddings_df['qid'] = idx

qids_val = val_embeddings_df.groupby('qid')['qid'].count().to_numpy()
X_val = val_embeddings_df.drop(['score'], axis = 1)

In [115]:
from sklearn.metrics import ndcg_score

y_pred = ranker.predict(val_embeddings_df.drop(['score'], axis=1))
y_pred = np.reshape(y_pred, (-1, 5))

y_true = val_embeddings_df['score']
y_true = np.reshape(y_true.values, (-1, 5))

print('VAL ndcg@1:', ndcg_score(y_true, y_pred, k=1))
print('VAL ndcg@3:', ndcg_score(y_true, y_pred, k=3))
print('VAL ndcg@5:', ndcg_score(y_true, y_pred, k=5))

VAL ndcg@1: 0.6458345153664303
VAL ndcg@3: 0.7296467874539512
VAL ndcg@5: 0.8622997633192735
